In [1]:
import pandas as pd 
import numpy as np

data = pd.read_csv('Processed Data Aus.csv')
data

,time,user_behavior,business_performance,value,business_reviews,business_rating,suburb_name,business_category_encoded,suburb_name_encoded
0,2022-01-13,0,1.0,0.25,0.443,0.954887,Aberfeldie,1,0.715931
1,2022-01-25,0,0.5,0.75,0.844,0.724311,Aberfeldie,17,0.715931
2,2022-01-30,0,0.0,1.00,0.447,0.328321,Aberfeldie,32,0.715931
3,2022-01-27,0,0.0,0.00,0.076,0.411028,Aberfeldie,35,0.715931
4,2022-01-24,0,0.5,0.75,0.655,0.764411,Aberfeldie,28,0.715931
...,...,...,...,...,...,...,...,...,...
480495,2022-01-23,0,1.0,0.75,0.100,0.433584,Yering,5,0.802303
480496,2022-01-10,1,1.0,0.25,0.047,0.796992,Yering,35,0.802303
480497,2022-01-28,0,0.0,0.00,0.957,0.456140,Yering,36,0.802303
480498,2022-01-16,1,0.5,0.75,0.486,0.596491,Yering,28,0.802303


In [2]:
grouped = data.groupby(["suburb_name_encoded"]).apply(lambda x: x.sort_values(["time"], ascending = True)).reset_index(drop=True)

In [3]:
grouped1 = grouped.drop(columns=['time','suburb_name'])

In [4]:
grouped1

,user_behavior,business_performance,value,business_reviews,business_rating,business_category_encoded,suburb_name_encoded
0,0,1.0,0.75,0.601,0.130326,9,0.0
1,1,1.0,0.50,0.346,0.095238,21,0.0
2,0,0.5,0.00,0.380,0.248120,11,0.0
3,0,1.0,0.75,0.601,0.130326,9,0.0
4,1,0.5,1.00,0.802,0.709273,22,0.0
...,...,...,...,...,...,...,...
480495,0,1.0,0.50,0.427,0.285714,46,1.0
480496,0,1.0,0.50,0.396,0.736842,31,1.0
480497,1,0.0,1.00,0.631,0.288221,17,1.0
480498,1,0.0,0.50,0.890,0.962406,5,1.0


In [5]:
grouped2 = grouped1.to_numpy()


In [6]:
grouped2

array([[ 0.        ,  1.        ,  0.75      , ...,  0.13032581,
         9.        ,  0.        ],
       [ 1.        ,  1.        ,  0.5       , ...,  0.0952381 ,
        21.        ,  0.        ],
       [ 0.        ,  0.5       ,  0.        , ...,  0.2481203 ,
        11.        ,  0.        ],
       ...,
       [ 1.        ,  0.        ,  1.        , ...,  0.28822055,
        17.        ,  1.        ],
       [ 1.        ,  0.        ,  0.5       , ...,  0.96240602,
         5.        ,  1.        ],
       [ 1.        ,  0.        ,  0.25      , ...,  0.93734336,
        12.        ,  1.        ]])

In [7]:
X, y = [], []
seq_len = 30
# for i in range(len(data) - seq_len):
for i in range(0, len(grouped2) - seq_len, seq_len-1):
    X.append(grouped2[i:i+seq_len, :-1])
    y.append(grouped2[i+seq_len-1, -1:])

X = np.array(X)
y = np.array(y)

# Print shapes of input and output arrays
print('Input shape:', X.shape)
print('Output shape:', y.shape)

Input shape: (16568, 30, 6)
Output shape: (16568, 1)


In [8]:
train_size = int(0.7 * len(X))
val_size = int(0.2 * len(X))
test_size = len(X) - train_size - val_size

train_X, val_X, test_X = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
train_y, val_y, test_y = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

print("Training data shape:", train_X.shape, train_y.shape)
print("Validation data shape:", val_X.shape, val_y.shape)
print("Test data shape:", test_X.shape, test_y.shape)

Training data shape: (11597, 30, 6) (11597, 1)
Validation data shape: (3313, 30, 6) (3313, 1)
Test data shape: (1658, 30, 6) (1658, 1)


In [9]:
train_X.shape[2]

6

In [ ]:
#!pip install -U tensorflow==2.8.2

In [10]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
# num_categories = len(df['Business category'].unique())
num_categories = 52
# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(512, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.MaxPooling1D(pool_size=2, strides=None, padding="valid"),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.MaxPooling1D(pool_size=2, strides=None, padding="valid"),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(208),# activation='relu'),
    tf.keras.layers.Dense(104),# activation='relu'),
    tf.keras.layers.Dense(num_categories)#, activation='softmax')
])

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_X, train_y,
                    validation_data=(val_X, val_y),
                    verbose=1,
                    shuffle=False, 
                    epochs=100,
                    batch_size=4096)

In [11]:
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from time import time

In [14]:
import tensorflow as tf

model1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(30, 6), return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(64, return_sequences =True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.LSTM(32, return_sequences =False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(208,activation='relu'),
    tf.keras.layers.Dense(104, activation ='relu'),
    tf.keras.layers.Dense(52, activation ='softmax')
    ])

# callbacks = [EarlyStopping(monitor='accuracy', patience = 15)]
optim = tf.keras.optimizers.Adam(learning_rate=0.001)

model1.compile(optimizer = optim,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

callbacks = [EarlyStopping(monitor='accuracy', patience=10)]

history = model1.fit(train_X, train_y, 
                     validation_split =0.15,
                     batch_size =64,
                     epochs =50,
                     #callbacks =callbacks,
                     verbose=1) 

Epoch 1/50


2023-04-27 12:34:58.785978: E tensorflow/stream_executor/cuda/cuda_dnn.cc:373] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-04-27 12:34:58.786006: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1556 : UNKNOWN: Fail to find the dnn implementation.


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_3/PartitionedCall]] [Op:__inference_train_function_18015]

In [13]:
train_X.shape[1], train_X.shape[2]

(30, 6)